In [1]:
import random
import numpy as np
import typing as tp

# graphs
import matplotlib.pyplot as plt
import networkx as nx
from networkx.generators.random_graphs import erdos_renyi_graph, complete_graph
from networkx.generators import circulant_graph
from scipy.stats import ortho_group

In [2]:
import sys
sys.path.append("../../")

from methods.minimization.base import BaseDecentralizedMethod
from methods.minimization import LoggerDecentralized, DecentralizedGD
from oracles.minimization import BaseSmoothOracle, OracleLinearComb, LinearRegressionL2Oracle

In [3]:
class DAPDG(BaseDecentralizedMethod):
    
    """
    oracle_list: list of size M of oracles for every node
    x_0: matrix of shape [M, N], must be if Range A^T
    B: matrix of affine constraints of shape [P, N]
    W: laplasian matrix of graph of shape [M, M]
    logger: haven't figured it out yet
    eta, alpha, beta, theta, sigma, gamma: parameters
    """
    def __init__(
            self,
            oracle_list: tp.List[BaseSmoothOracle],
            x_0: np.ndarray,
            B: np.ndarray,
            W: np.ndarray,
            logger: LoggerDecentralized,
            gamma: float = 0.75
    ):
        super().__init__(oracle_list, x_0, logger)
                
        self.B = B
        self.W = W
        
        M = W.shape[0]
        P, D = B.shape
        self.D, self.M = D, M
        
        assert W.shape == (M, M)
        assert self.x.shape == (M, D)
        
        self.gamma = gamma  # parameter for matrix A

        bold_B = np.kron(np.ones((M, M)), B)
        bold_W = np.kron(W, np.ones((D, D)))
        self.A = np.vstack([bold_B, gamma * bold_W])
        
        L_mu_params = self.compute_l_mu(oracle_list, self.A)
        self.init_parameters(**L_mu_params)
        assert self.eta > 0 and self.alpha > 0 and self.beta > 0
        assert 0 < self.tau <= 1 and 0 < self.sigma <= 1
        
        self.x_f = np.zeros((M, D))
    
        
    def compute_l_mu(self, oracle_list, mat):
        oracle = OracleLinearComb(oracle_list, [1 / len(oracle_list) for _ in oracle_list])
        l_x = max([np.linalg.svd(oracle.A)[1].max() / oracle.den + oracle.regcoef for oracle in oracle_list])
        mu_x = min([oracle.regcoef for oracle in oracle_list])
        sing_vals = np.linalg.svd(mat)[1]
        l_xy = sing_vals.max()**2
        mu_xy = sing_vals[sing_vals >= 1e-10].min()**2
        return dict(L_x=l_x, mu_x=mu_x, L_xy=l_xy, mu_xy=mu_xy)
        
    
    def init_parameters(self, mu_x, mu_xy, L_x, L_xy):
        delta = (mu_xy**2 / (2*mu_x*L_x))**.5
        rho_b = (4 + 8 * max(L_xy / mu_xy * (L_x / mu_x)**.5,
                             L_xy**2 / mu_xy**2)) ** (-1)
        self.sigma = (mu_x / (2 * L_x))**.5
        self.alpha = mu_x
        # self.theta = 1 - rho_b
        self.eta = min(1 / (4 * (mu_x + L_x * self.sigma)), delta / (4 * L_xy))
        self.beta = 1 / (2 * self.eta * L_xy ** 2)
        self.tau = (self.sigma**(-1) + 0.5)**(-1)
        

    def step(self):
        A = self.A
        x = self.x.reshape(self.M * self.D)
        x_f = self.x_f.reshape(self.M * self.D)
        x_g = self.tau * x + (1 - self.tau) * x_f
        
        grad_F_x_g = self.grad_list(x_g.reshape(M, D)).reshape(D * M)  # np.array[D * M]
        
        x_new = x + self.eta * self.alpha * (x_g - x) - \
            self.eta * (self.beta * A.T.dot(A.dot(x)) - grad_F_x_g)
        
        x_f = x_g + self.sigma * (x_new - x)
        
        self.x = x_new.reshape(self.M, self.D)
        self.x_f = x_f.reshape(self.M, self.D)
        
        
    """
    W: laplasian matrix of graph of shape [M, M]
    """
    def set_new_graph(self, W):
        assert W.shape == (self.M, self.M)
        bold_B = np.kron(np.ones((self.M, self.M)), self.B)
        bold_W = np.kron(W, np.ones((self.N, self.N)))
        self.A = np.vstack([bold_B, self.gamma * bold_W])

        
def create_matrix_with_L_mu(nx, ny, L_xy, mu_xy):
    rvsx = lambda: ortho_group.rvs(dim=nx)
    rvsy = lambda: ortho_group.rvs(dim=ny)

    nA = min(nx, ny)
    eigvals_A = np.linspace(mu_xy, L_xy, nA)
    if nx < ny:
        mA = np.concatenate((np.diag(eigvals_A), np.zeros([nx, ny - nA])), axis=1)
    else:
        mA = np.concatenate((np.diag(eigvals_A), np.zeros([nx - nA, ny])), axis=0)
    A = rvsx().dot(mA).dot(rvsy()).T

    return A

In [4]:
D = 3  # dimension of space 
M = 5  # count of nodes
P = 4  # dimension of constraints
L, mu = 10, 1

In [5]:
x = np.random.rand(M, D)
B = create_matrix_with_L_mu(P, D, L, mu).T
B = np.random.rand(P, D)
G = circulant_graph(M, [1])
W = nx.laplacian_matrix(G).A  # may be better not to convert into numpy array

oracle_list = [LinearRegressionL2Oracle(np.random.rand(D, D), np.random.rand(D), 1) for _ in range(M)]
logger = LoggerDecentralized()

<class 'networkx.utils.decorators.argmap'> compilation 8:4: FutureWarning: laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


In [6]:
opt = DAPDG(oracle_list, x, B, W, logger)
opt.run(max_iter=1000)
opt.x

array([[-8.3868059 , -8.74748731, -4.15884293],
       [-2.01957109, -2.59672107,  1.8435837 ],
       [11.97317479, 14.84356942,  0.65668091],
       [ 9.43584105,  2.68357677,  7.35202503],
       [-4.38853252, -3.25289342, -5.40119519]])